In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import lightgbm as lgb

import shap

In [3]:
# Loading Data
heart_df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')

### Exploratory Data Analysis

In [4]:
print(heart_df.shape)
heart_df.head()

In [5]:
heart_df.info()

In [6]:
for c in heart_df.columns:
    if heart_df[c].dtypes == 'O':
        print('Column:', c)
        print(heart_df[c].value_counts())
        print('-------------------------------------')

In [7]:
heart_df.describe()

In [8]:
heart_df.hist(bins=20, figsize=(20,15))

In [9]:
# Correlation Matrix
corr = heart_df.corr()

sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)
corr

### Data Preprocessing

In [10]:
y_column = ['HeartDisease']
X = heart_df[[c for c in heart_df.columns if c not in y_column]]
y = heart_df[y_column]

In [11]:
print(X.shape)
print(X.columns)
X.head()

In [12]:
cat_attributes = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
num_attribtes = [c for c in X.columns if c not in cat_attributes]

In [13]:
X_num = X[num_attribtes]
X_cat = X[cat_attributes]

In [14]:
cat_encoder = OneHotEncoder()
num_scaler = StandardScaler()

In [15]:
X_cat = pd.get_dummies(X_cat, drop_first=True)

In [16]:
X_num = num_scaler.fit_transform(X_num)
X_num = pd.DataFrame(X_num, columns=num_attribtes)

In [17]:
X_prepared = pd.concat([X_cat, X_num], axis=1)

In [18]:
print(X_prepared.shape)
X_prepared.head()

### Modeling

#### Train - Test Split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_prepared, y, test_size = 0.25, random_state = 42)

In [20]:
print('Train Data Size:', X_train.shape)
print('Test Data Size:', X_test.shape)

In [21]:
print('Label Distribution in training data:', y_train[y_column].value_counts())
print('Label Distribution in test data:', y_test[y_column].value_counts())

#### Logistic Regression

In [22]:
log_reg_model = LogisticRegression(random_state=42, penalty='l2', C=0.1)
log_reg_model.fit(X_train, y_train)

In [23]:
y_pred = log_reg_model.predict(X_test)
y_pred_prob = log_reg_model.predict_proba(X_test)

In [24]:
print('Accuracy Score:', accuracy_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))
print('AUC:', roc_auc_score(y_test, y_pred))

In [25]:
X_train.columns

In [26]:
print('Logistic Model Coefficients:')
for i, j in zip(X_train.columns, log_reg_model.coef_[0]):
    print(i, round(j,4))

sns.set(rc={'figure.figsize':(20, 8)})

p = sns.barplot(x = X_train.columns, y = log_reg_model.coef_[0])

#### Light GBM Model

In [27]:
dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature='auto')
dtest = lgb.Dataset(X_test, label=y_test, categorical_feature='auto')

In [28]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'max_cat_to_onehot': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'seed': 100
}

In [29]:
gbm = lgb.train(params,
                dtrain,
                num_boost_round=100,
                verbose_eval=100,
                valid_sets=[dtest, dtrain],  # eval training data
                )

In [30]:
y_pred = gbm.predict(X_test)
print(y_pred[:5])
print(y_pred.shape)

In [31]:
f, axs = plt.subplots(1, 2, figsize=(25, 5))
lgb.plot_importance(gbm, importance_type='gain', title='Feature Importance(gain)', ax=axs[0]);
lgb.plot_importance(gbm, importance_type='split', title='Feature Importance(split)', ax=axs[1]);

### Computing SHAP values

#### Kernel SHAP

In [32]:
logit_explainer = shap.KernelExplainer(log_reg_model.predict_proba, X_train)

In [2]:
shap_values_single = logit_explainer.shap_values(X_test.iloc[0:100,:], nsamples=1000)

In [44]:
shap.initjs()
shap.force_plot(logit_explainer.expected_value[0], shap_values_single[0], X_test.iloc[0:100,:])

In [47]:
shap.summary_plot(shap_values_single[0], X_test.iloc[0:100,:])

#### Tree SHAP

In [ ]:
explainer = shap.TreeExplainer(model=gbm,
                               data=None,
                               model_output='raw',
                               feature_perturbation='tree_path_dependent')

In [ ]:
shap_values = explainer.shap_values(X_test)

In [ ]:
print(f'Shape of test dataset: {X_test.shape}')
print(f'Type of shap_values: {type(shap_values)}. Length of the list: {len(shap_values)}')
print(f'Shape of shap_values: {np.array(shap_values).shape}')

#### Explaining a single prediction

In [ ]:
print(y_test.head(1))
X_test.head(1)


In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], features=X_test.iloc[0, :])

In [ ]:
print(f'True value for 0-th observation: {y_test.iloc[0]}')
print(f'Probability of default for 0-th observation: {y_pred[0]}')
print(f'Raw model prediction (logit) for 0-th observation: {gbm.predict(X_test, raw_score=True)[0]}')
print(f'Base value = mean logit (log-odds) over training data = {np.mean(gbm.predict(X_train, raw_score=True))}')

In [ ]:
print(f'Base value is stored as an attribute of the explainer object: {explainer.expected_value}')

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][:100,:], features=X_test.iloc[:100, :])

In [ ]:
shap.summary_plot(shap_values, features=X_test)

In [ ]:
shap.summary_plot(shap_values, features=X_test, class_inds=[1])

In [ ]:
shap.summary_plot(shap_values[1], features=X_test)